In [1]:
import requests
import bs4

print(requests.__version__)
print(bs4.__version__)

2.31.0
4.12.2


In [2]:
URL = 'https://www.naver.com/'

req = requests.get(URL)
print(req.status_code)

200


In [3]:
# req.text

In [36]:
from bs4 import BeautifulSoup
import pandas as pd 
with open('index.html', 'r', encoding='UTF8') as f:
    
    # step 01: 데이터 수집
    contents = f.read()
    
    # step 02: 데이터 파싱 (순수한 HTML 파일을 BeautifulSoup 객체로 변환)
    soup = BeautifulSoup(contents, 'lxml')
    #print(soup)
    
    #print(soup.h2)
    #print(soup.ul)
    #print("-----")
    #print(soup.ul.li)
    # 4개의 li 태그에 있는 회사명을 모두 가져오는 것이 목적
    
    # step 03: 데이터 수집을 위한 특정 태그 찾기
    companies = []
    print(soup.find_all('li'))
    
    # step 04: 데이터 가공
    for tag in soup.find_all('li'):
        companies.append(tag.text)
    print(companies)
    
    # step 05: 처리된 데이터 저장 pandas 데이터프레임
    crawling_dict = {'회사명': companies}
    result = pd.DataFrame(crawling_dict)
    print(result)
    
    # step 06: csv 파일로 내보내기 or DB로 내보내기
    result.to_csv("result.csv", index=False)

[<li>애플</li>, <li>삼성</li>, <li>노키아</li>, <li>LG</li>]
['애플', '삼성', '노키아', 'LG']
   회사명
0   애플
1   삼성
2  노키아
3   LG


In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 

def crawling(soup) :
    # print(soup)
    tbody = soup.find("tbody")
    result = []
    for p in tbody.find_all('p', class_ = 'title'):
        result.append(p.get_text().strip())
    return result

def main() :
    custom_header = {
        'referer' : 'https://music.bugs.co.kr/',
        'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }
    url = "https://music.bugs.co.kr/chart" # 크롤링 하려는 웹사이트
    req = requests.get(url, headers = custom_header)
    
    soup = BeautifulSoup(req.text, "html.parser")
    crawling(soup)

    titles = crawling(soup)
    print(pd.DataFrame({"노래제목" : titles}))


if __name__ == "__main__" :
    main()

                노래제목
0      Love wins all
1               Wife
2              To. X
3           Love 119
4               에피소드
..               ...
95    When I Get Old
96             밤, 바다
97           사랑을 하다가
98    안 아름답고도 안 아프구나
99  그대가 있는 곳, 언제 어디든

[100 rows x 1 columns]


In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import warnings
import time
import random
warnings.filterwarnings('ignore') 

def crawling(url, headers, soup):
    last_page = int(soup.select_one('td.pgRR').a['href'].split('=')[-1])
    
    df = None
    count = 0
    for page in range(1, last_page + 1):
      req = requests.get(f'{url}&page={page}', headers=headers)
      df = pd.concat([df, pd.read_html(req.text, encoding = "euc-kr")[0]], ignore_index=True)
      if count > 10:
        break
      count += 1
      time.sleep( random.uniform(2,4)) 

    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    
    return df

def main():
    company_code = '005930' # 삼성전자
    url ="https://finance.naver.com/item/sise_day.nhn?code=" + company_code
    
    headers = { 
             'referer' : 'https://finance.naver.com/item/sise.naver?code=005930',
             'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
            }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    result = crawling(url, headers, soup)
    print(result)

if __name__ == "__main__":
    main()


             날짜       종가     전일비       시가       고가       저가         거래량
0    2024.02.01  73800.0  1100.0  73000.0  73800.0  72900.0   5009174.0
1    2024.01.31  72700.0  1600.0  73400.0  74000.0  72500.0  15703560.0
2    2024.01.30  74300.0   100.0  75000.0  75300.0  73700.0  12244418.0
3    2024.01.29  74400.0  1000.0  73800.0  75200.0  73500.0  13976521.0
4    2024.01.26  73400.0   700.0  73700.0  74500.0  73300.0  11160062.0
..          ...      ...     ...      ...      ...      ...         ...
115  2023.08.11  67500.0   500.0  68400.0  68800.0  67500.0   9781038.0
116  2023.08.10  68000.0   900.0  68300.0  68500.0  67800.0  10227311.0
117  2023.08.09  68900.0  1300.0  68000.0  69600.0  67900.0  17259673.0
118  2023.08.08  67600.0   900.0  69000.0  69100.0  67400.0  14664709.0
119  2023.08.07  68500.0   200.0  67700.0  69200.0  67600.0  10968505.0

[120 rows x 7 columns]
